## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-21-10-48-25 +0000,bbc,US chess grandmaster Daniel Naroditsky dies ag...,https://www.bbc.com/news/articles/c15pz8vpjp9o...
1,2025-10-21-10-48-07 +0000,nyt,"Nations Hesitate to Send Troops to Gaza, Feari...",https://www.nytimes.com/2025/10/21/world/middl...
2,2025-10-21-10-40-08 +0000,bbc,Woman charged after 1.5m euros of gold stolen ...,https://www.bbc.com/news/articles/c62ez36ndp3o...
3,2025-10-21-10-37-20 +0000,cbc,Man who admitted college campus sexual assault...,https://www.cbc.ca/news/world/us-gettysburg-se...
4,2025-10-21-10-27-28 +0000,bbc,Tornado hits Paris suburbs leaving one dead,https://www.bbc.com/news/videos/c9864114nemo?a...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2319/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
149,trump,39
188,louvre,15
357,new,13
209,china,12
10,gaza,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
76,2025-10-21-04-55-46 +0000,nypost,Student loan debt will be forgiven for million...,https://nypost.com/2025/10/21/us-news/student-...,97
177,2025-10-20-20-06-32 +0000,latimes,Trump again suggests sending troops to San Fra...,https://www.latimes.com/california/story/2025-...,94
131,2025-10-20-23-05-53 +0000,nypost,"Trump can deploy National Guard to Portland, O...",https://nypost.com/2025/10/20/us-news/9th-circ...,92
125,2025-10-20-23-45-15 +0000,nyt,Appeals Court Ruling Allows Trump to Deploy Na...,https://www.nytimes.com/2025/10/20/us/appeals-...,89
162,2025-10-20-21-06-07 +0000,bbc,Appeals court allows Trump to deploy National ...,https://www.bbc.com/news/articles/c201k3qky9lo...,82


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
76,97,2025-10-21-04-55-46 +0000,nypost,Student loan debt will be forgiven for million...,https://nypost.com/2025/10/21/us-news/student-...
25,63,2025-10-21-09-48-55 +0000,nyt,Vance Lands in Israel as U.S. Tries to Shore U...,https://www.nytimes.com/2025/10/21/world/middl...
131,53,2025-10-20-23-05-53 +0000,nypost,"Trump can deploy National Guard to Portland, O...",https://nypost.com/2025/10/20/us-news/9th-circ...
232,43,2025-10-20-15-10-02 +0000,nypost,French authorities admit they ‘failed’ in Louv...,https://nypost.com/2025/10/20/world-news/louvr...
77,43,2025-10-21-04-53-16 +0000,wapo,"Japan elects first female leader, a hawk who p...",https://www.washingtonpost.com/world/2025/10/2...
94,36,2025-10-21-02-08-08 +0000,bbc,US and Australia sign rare earths deal to coun...,https://www.bbc.com/news/articles/cly9kvrdk2xo...
156,33,2025-10-20-21-18-51 +0000,bbc,Former NFL player Doug Martin dies in police c...,https://www.bbc.com/news/articles/c6297qm94z0o...
268,32,2025-10-20-11-17-29 +0000,cbc,Hamas turns over another hostage body after Is...,https://www.cbc.ca/news/world/gaza-israel-medi...
185,31,2025-10-20-19-29-11 +0000,eltecolote,‘Coco’ at the SF Symphony: Send a photo of you...,https://eltecolote.org/content/en/dia-de-los-m...
111,30,2025-10-21-00-53-48 +0000,nypost,Curtis Sliwa’s checkered past catches up to hi...,https://nypost.com/2025/10/20/us-news/curtis-s...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
